### 1. Preparando e limpando os dados

Preparando o ambiente com as bibliotecas necessárias.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

Para garantir que os arquivos .csv estão corretos, eu fiz uma alteração direta nos arquivos, mudando o separador para vírgula para garantir que as colunas fossem separadas corretamente.

In [ ]:
customers = "customers.csv"
orders = "orders.csv"

customers_df = pd.read_csv(customers)
orders_df = pd.read_csv(orders)

In [ ]:

print("Customers dataframe info:")
print(customers_df.info())
print("\n")
print("Orders dataframe info:")
print(orders_df.info())
print("\n")

print("First 5 rows of customers:")
display(customers_df.head())
print("\n")
print("First 5 rows of orders:")
display(orders_df.head())

Tendo a certeza de que a database está devidamente carregada, vamos ver algumas estatísticas como médias, mediana, desvio padrão, etc., para entender melhor os dados e, se necessário, limpar os outliers ou dados inválidos. 

Também é interessante plotar para entender a distribuição e se será necessário aplicar alguma transformação.

Os percentis serão calculados aqui, com o método .describe().

In [ ]:
print("Statistical info about orders:")
#Os percentis estão calculados aqui.
print(orders_df.describe())
print("\n")

# Plot da distribuição das quantidades.
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(data=orders_df, x='quantity', kde=True, bins=range(1, int(orders_df['quantity'].max())+2))
plt.title('Distribution of order quantities')
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.xticks(range(1, int(orders_df['quantity'].max())+1))

# Plot da distribuição de preços.
plt.subplot(1, 2, 2)
sns.histplot(data=orders_df, x='price', kde=True)
plt.title('Distribution of order prices')
plt.xlabel('Price')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

### 2. Números e dados

Receita por país e receita total.

In [ ]:
revenue_by_country = orders_df.merge(customers_df, on='customer_id', how='left')

revenue_by_country = revenue_by_country.groupby('country')['price'].sum().reset_index()
revenue_by_country = revenue_by_country.sort_values('price', ascending=False)

print(revenue_by_country)

total_revenue = revenue_by_country['price'].sum()
print(f"\nReceita total: ${total_revenue:,.2f}")

Valor médio de pedido, também dividido por país.

In [ ]:
avg_order_value = orders_df.merge(customers_df, on='customer_id', how='left')
avg_order_value = avg_order_value.groupby('country')['price'].mean().reset_index()
avg_order_value = avg_order_value.sort_values('price', ascending=False)

print(avg_order_value)

overall_avg = avg_order_value['price'].mean()
print(f"\nValor médio de uma order: ${overall_avg:,.2f}")

Pedidos por país e pedidos totais.

In [ ]:
orders_per_country = orders_df.merge(customers_df, on='customer_id', how='left')
orders_per_country = orders_per_country.groupby('country').size().reset_index(name='number_of_orders')
orders_per_country = orders_per_country.sort_values('number_of_orders', ascending=False)

print(orders_per_country)

total_orders = orders_per_country['number_of_orders'].sum()
print(f"\nTotal de orders: {total_orders}")

Top-5 produtos por receita.

In [ ]:
product_revenue = orders_df.groupby('product_id')['price'].sum().reset_index()
product_revenue = product_revenue.sort_values('price', ascending=False)

top_5_products = product_revenue.head(5)

print(top_5_products)

Top-3 clientes por receita.

In [ ]:
customer_revenue = orders_df.groupby('customer_id')['price'].sum().reset_index()
customer_revenue = customer_revenue.sort_values('price', ascending=False)

top_3_customers = customer_revenue.head(3)
top_3_customers = top_3_customers.merge(customers_df.drop_duplicates(subset='customer_id'),  on='customer_id', how='left')

print(top_3_customers[['customer_id', 'name', 'email', 'price']])

### 3. Armazenamento e finalização do arquivo

In [ ]:
analysis_results = {}

analysis_results['revenue_analysis'] = {
    'by_country': revenue_by_country.to_dict('records'),
    'total_revenue': float(round(total_revenue, 2))
}

analysis_results['average_order_value'] = {
    'by_country': avg_order_value.round(2).to_dict('records'),
    'overall_average': float(round(overall_avg, 2))
}

analysis_results['order_analysis'] = {
    'orders_by_country': orders_per_country.to_dict('records'),
    'total_orders': int(total_orders)
}

analysis_results['product_analysis'] = {
    'top_products': top_5_products.round(2).to_dict('records')
}

analysis_results['customer_analysis'] = {
    'top_customers': top_3_customers[['customer_id', 'name', 'email', 'price']].round(2).to_dict('records')
}

import json
print(json.dumps(analysis_results, indent=2))


In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):

            if v and isinstance(v[0], dict):
                items.append((new_key, json.dumps(v)))
            else:
                items.append((new_key, str(v)))
        else:
            items.append((new_key, v))
    return dict(items)

flattened_data = flatten_dict(analysis_results)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
flattened_data['generated_at'] = current_time

df_results = pd.DataFrame([flattened_data])
df_results.to_csv('results_2023.csv', index=False)

print("Results saved to results_2023.csv")